In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
moment_hsi = []
for i in range(3):
    for j in range(4):
        img_path = "moment/pc_" + str(i) + "_" + str(j) + ".png"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        moment_hsi.append(img)
        
moment_hsi = np.array(moment_hsi)

In [0]:
moment_lidar = []
for i in range(4):
    img_path = "moment/lidar_" + str(i) + ".png"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    moment_lidar.append(img)

moment_lidar = np.array(moment_lidar)

In [44]:
print(moment_hsi.shape)
print(moment_lidar.shape)

(12, 349, 1905)
(4, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
moment_hsi_train_data = []
moment_hsi_test_data = []
moment_lidar_train_data = []
moment_lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            moment_hsi_train_data.append(moment_hsi[:, i, j])
            moment_lidar_train_data.append(moment_lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            moment_hsi_test_data.append(moment_hsi[:, i, j])
            moment_lidar_test_data.append(moment_lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
moment_hsi_train_data = np.array(moment_hsi_train_data)
moment_hsi_test_data = np.array(moment_hsi_test_data)
moment_lidar_train_data = np.array(moment_lidar_train_data)
moment_lidar_test_data = np.array(moment_lidar_test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [49]:
print(moment_hsi_train_data.shape)
print(moment_hsi_test_data.shape)
print(moment_lidar_train_data.shape)
print(moment_lidar_test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 12)
(12197, 12)
(2832, 4)
(12197, 4)
(2832,)
(12197,)


In [0]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

In [51]:
print(train_one_hot.shape)
print(test_one_hot.shape)
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)
(2832, 15)
(12197, 15)


In [0]:
HSI_PATCH_SIZE = 27
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005
BATCH_SIZE = 25

In [0]:
import numpy as np
padded_moment_hsi = np.lib.pad(moment_hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
padded_moment_lidar = np.lib.pad(moment_lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')

In [54]:
print(padded_moment_hsi.shape)
print(padded_moment_lidar.shape)

(12, 375, 1931)
(4, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
moment_hsi_train_patches = []
moment_hsi_test_patches = []
moment_lidar_train_patches = []
moment_lidar_test_patches = []

for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      moment_hsi_train_patches.append(get_patches(padded_moment_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      moment_lidar_train_patches.append(get_patches(padded_moment_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      moment_hsi_test_patches.append(get_patches(padded_moment_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      moment_lidar_test_patches.append(get_patches(padded_moment_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [0]:
moment_hsi_train_patches = np.array(moment_hsi_train_patches)
moment_hsi_test_patches = np.array(moment_hsi_test_patches)
moment_lidar_train_patches = np.array(moment_lidar_train_patches)
moment_lidar_test_patches = np.array(moment_lidar_test_patches)

In [58]:
print(moment_hsi_train_patches.shape)
print(moment_hsi_test_patches.shape)
print(moment_lidar_train_patches.shape)
print(moment_lidar_test_patches.shape)

(2832, 27, 27, 12)
(12197, 27, 27, 12)
(2832, 41, 41, 4)
(12197, 41, 41, 4)


In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = moment_hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

moment_hsi_model = Sequential()
moment_hsi_model.add(InputLayer(input_shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS)))
moment_hsi_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
moment_hsi_model.add(BatchNormalization())
moment_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))
moment_hsi_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
moment_hsi_model.add(BatchNormalization())
moment_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))


moment_hsi_model.add(Flatten())
moment_hsi_model.add(Dense(FC1, activation='relu'))
moment_hsi_model.add(Dropout(0.6))
moment_hsi_model.add(Dense(FC2, activation='relu'))
moment_hsi_model.add(Dropout(0.4))
moment_hsi_model.add(Dense(NUM_CLS, activation='softmax'))

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = moment_lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


moment_lidar_model = Sequential()
moment_lidar_model.add(InputLayer(input_shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS)))
moment_lidar_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
moment_lidar_model.add(BatchNormalization())
moment_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))
moment_lidar_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
moment_lidar_model.add(BatchNormalization())
moment_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))


moment_lidar_model.add(Flatten())
moment_lidar_model.add(Dense(FC1, activation='relu'))
moment_lidar_model.add(Dropout(0.7))
moment_lidar_model.add(Dense(FC2, activation='relu'))
moment_lidar_model.add(Dropout(0.5))
moment_lidar_model.add(Dense(NUM_CLS, activation='softmax'))


In [0]:
moment_hsi_model.load_weights('Models/moment_hsi_model_weights.h5')
moment_lidar_model.load_weights('Models/moment_lidar_model_weights.h5')

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [63]:
test_cls = test_labels - 1
prediction = moment_hsi_model.predict(moment_hsi_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing


overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of testing sapmles : ",overall_acc)
print("Average Accuracy of testing samples : ",average_acc)
print("Kappa statistics of testing samples : ",kappa)
print("Each class accuracy of testing samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of testing sapmles :  0.615069279330983
Average Accuracy of testing samples :  0.6164649438319929
Kappa statistics of testing samples :  0.5822158999518787
Each class accuracy of testing samples :  [0.79722519 0.68808411 0.89675516 0.69969666 0.84215686 0.42913386
 0.76692456 0.320059   0.62756849 0.7629108  0.57047478 0.47007397
 0.41904762 0.49069767 0.46616541]
Confusion matrix : [[747  94   0  75   0   0  17   1   0   0   0   1   0   0   2]
 [ 77 589   0 166   0   1  10   2   4   0   0   0   0   0   7]
 [  0   0 304   0   0   0   1   4   0   6   0   0   5   0  19]
 [ 47 162   0 692   0   0  59   0   0   0   0  23   0   0   6]
 [  0   1   0   1 859   0   0   3  27   0   0  43   0  23  63]
 [  4   4  98   2   0 109   5   6   0   0   0   7   0   0  19]
 [  0  21   7  24   7  21 793  13  90   0   0   4  25   0  29]
 [154 164   3  77  43   3  40 434  61 117  46 130   6   0  78]
 [  0   6   0   4  57   4  73  66 733  23  20   0   3   1 178]
 [  0   0   9   0   0   0   0 

In [64]:
test_cls = test_labels - 1
prediction = moment_lidar_model.predict(moment_lidar_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing


overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of testing sapmles : ",overall_acc)
print("Average Accuracy of testing samples : ",average_acc)
print("Kappa statistics of testing samples : ",kappa)
print("Each class accuracy of testing samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of testing sapmles :  0.4681479052225957
Average Accuracy of testing samples :  0.4493144653171762
Kappa statistics of testing samples :  0.4240807232116879
Each class accuracy of testing samples :  [0.265625   0.49285714 0.36330935 0.48461538 0.54476909 0.32748538
 0.52159091 0.40258065 0.22570726 0.66828322 0.71980279 0.5492228
 0.30448718 0.44444444 0.42493639]
Confusion matrix : [[102  22   0  16  11   1  26  32  34  39   4  89   0   4   4]
 [  5 276  62  42  13   0   4  13  13  34   0  24   6   0  68]
 [102  28 202  11  14   0   1   0  41   0   0  42   6   0 109]
 [  7   2   0 315   0   0 123  80  46  60   0   5   2   0  10]
 [144  32   0   8 578   0  52   1 177  11  10  26  11   2   9]
 [ 45   1   0  94   9 112   1   8  12   0   0  60   0   0   0]
 [  0  13   0 184   0   0 459 122  41  19   0  11   0   0  31]
 [ 42 102   0 254 251   0 161 624  61  17   0  24   0   6   8]
 [353  91 149  38 125   6 122   3 367 113  95  45  70   1  48]
 [ 63 120   0  10  16   0  15 

In [0]:
from tensorflow.python.keras.models import Model
intermediate_layer_moment_hsi_model = Model(moment_hsi_model.layers[0].input,outputs=moment_hsi_model.layers[6].output)
intermediate_layer_moment_lidar_model = Model(moment_lidar_model.layers[0].input,outputs=moment_lidar_model.layers[6].output)

moment_hsi_train_flatten = intermediate_layer_moment_hsi_model.predict(moment_hsi_train_patches)
moment_lidar_train_flatten = intermediate_layer_moment_lidar_model.predict(moment_lidar_train_patches)

moment_hsi_test_flatten = intermediate_layer_moment_hsi_model.predict(moment_hsi_test_patches)
moment_lidar_test_flatten = intermediate_layer_moment_lidar_model.predict(moment_lidar_test_patches)

In [66]:
print(moment_hsi_train_flatten.shape)
print(moment_lidar_train_flatten.shape)
print(moment_hsi_test_flatten.shape)
print(moment_lidar_test_flatten.shape)

(2832, 400)
(2832, 400)
(12197, 400)
(12197, 400)


In [67]:
train_fusion = np.concatenate((moment_hsi_train_flatten, moment_lidar_train_flatten), axis=1)
test_fusion = np.concatenate((moment_hsi_test_flatten, moment_lidar_test_flatten), axis=1)
print(train_fusion.shape)
print(test_fusion.shape)

(2832, 800)
(12197, 800)


In [68]:
from tensorflow.keras.layers import concatenate
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD


fusion_model = Sequential()
fusion_model.add(Dense(512, input_dim=train_fusion.shape[1], activation='relu'))

fusion_model.add(Dropout(0.8))
fusion_model.add(Dense(256, activation='relu'))
fusion_model.add(Dropout(0.7))
fusion_model.add(Dense(128, activation='relu'))
fusion_model.add(Dropout(0.6))
fusion_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE/2, decay=1e-6, momentum=0.9, nesterov=True)
fusion_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

fusion_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               410112    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 15)                1935      
Total para

In [93]:
history = fusion_model.fit(train_fusion, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 1s 195us/step - loss: 0.0572 - acc: 0.9841
Epoch 2/50
2832/2832 [==============================] - 1s 195us/step - loss: 0.0591 - acc: 0.9823
Epoch 3/50
2832/2832 [==============================] - 1s 197us/step - loss: 0.0569 - acc: 0.9848
Epoch 4/50
2832/2832 [==============================] - 1s 185us/step - loss: 0.0623 - acc: 0.9852
Epoch 5/50
2832/2832 [==============================] - 1s 195us/step - loss: 0.0535 - acc: 0.9869
Epoch 6/50
2832/2832 [==============================] - 1s 194us/step - loss: 0.0936 - acc: 0.9792
Epoch 7/50
2832/2832 [==============================] - 1s 199us/step - loss: 0.0586 - acc: 0.9834
Epoch 8/50
2832/2832 [==============================] - 1s 200us/step - loss: 0.0572 - acc: 0.9841
Epoch 9/50
2832/2832 [==============================] - 1s 191us/step - loss: 0.0896 - acc: 0.9799
Epoch 10/50
2832/2832 [==============================] - 1s 194us/step - loss: 0.0805 - acc: 0.9802
Epoch 11/

In [94]:
test_cls = test_labels - 1
prediction = fusion_model.predict(test_fusion).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.6292530950233663
Average Accuracy of training samples :  0.6344103835627434
Kappa statistics of training samples :  0.5989034435837433
Each class accuracy of training samples :  [0.83882784 0.6038688  0.76890756 0.83068783 0.81549815 0.55445545
 0.8204182  0.61845387 0.53616637 0.888      0.59518435 0.65782933
 0.30626058 0.51435407 0.16724337]
Confusion matrix : [[687   7   0  56   3   1  14   0   0   0   0  27   0   0  24]
 [ 57 718  68 132   0   0  59   0   7   0   1  28   0   0 119]
 [  2   6 183  10   0   0   0   0   5   0   0   0   1   0  31]
 [ 31   6   0 628   0   0  82   0   1   0   0   8   0   0   0]
 [ 29  24   0   6 884   0   0   0 102   0   0  25   0   3  11]
 [ 29   4   0  25   0 112  13   0   0   0   0  11   7   0   1]
 [  0   1   0  39   0   0 667  22  54  20   3   0   4   0   3]
 [ 99 123   0  65  36   0  87 744   2   6   0  31   0   1   9]
 [  7  28 126   1 119   5  54  19 593  59  11   1   4   0  79]
 [  2   0   1   0   7   0

In [0]:
fusion_model.save_weights('Models/moment_fusion_model_weights.h5')